In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [ ]:
class Graphstate(TypedDict):
    department: str
    message: str

In [ ]:
def router(state: Graphstate):
    department = state["department"].lower()
    if state["department"] == department:
        return "transfer_to_devops"
    elif state["department"] == department:
        return "transfer_to_hardware"
    
def devops_department(state: Graphstate):
    reply = f"Reply to your message {state["message"]}\nI'll reply the "